In [20]:
import os
import yaml
import json
import re
import streamlit as st
from langchain.prompts import PromptTemplate
from langchain.schema import HumanMessage
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI

model_name = 'gpt-3.5-turbo-1106'

In [21]:
import os
from dotenv import load_dotenv


# Load environment variables from .env file
load_dotenv()

user_api_key  = os.getenv('API_KEY')


In [22]:
llm = ChatOpenAI(model_name=model_name, openai_api_key=user_api_key, temperature=0)


In [23]:
def get_data_overview(df):
    """
    Obtain the shape, head, nunique, and description information of the DataFrame.
    """
    shape_info = str(df.shape)
    head_info = df.head().to_csv()
    nunique_info = df.nunique().to_csv()
    description_info = df.describe(include='all').to_csv()
    return shape_info, head_info, nunique_info, description_info

In [24]:
import pandas as pd
df = pd.read_csv('../dataset/VN_housing_dataset.csv')
df.head()

,Unnamed: 0,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
0,0,8/5/2020,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4,5 phòng,46 m²,NaN,NaN,"86,96 triệu/m²"
1,1,8/5/2020,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",NaN,NaN,3 phòng,37 m²,NaN,NaN,"116,22 triệu/m²"
2,2,8/5/2020,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4,4 phòng,40 m²,10 m,4 m,65 triệu/m²
3,3,8/5/2020,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,6 phòng,51 m²,12.75 m,4 m,100 triệu/m²
4,4,8/5/2020,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",NaN,NaN,4 phòng,36 m²,9 m,4 m,"86,11 triệu/m²"


In [25]:
df.columns

Index(['Unnamed: 0', 'Ngày', 'Địa chỉ', 'Quận', 'Huyện', 'Loại hình nhà ở',
       'Giấy tờ pháp lý', 'Số tầng', 'Số phòng ngủ', 'Diện tích', 'Dài',
       'Rộng', 'Giá/m2'],
      dtype='object')

In [26]:
def preprocessing(df):
    # Drop Unname column
    df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

    column_type = {
        "date" : ["Ngày"],
        "string": ['Địa chỉ','Quận','Huyện','Loại hình nhà ở','Giấy tờ pháp lý'],
        "number": ["Số tầng","Số phòng ngủ","Diện tích",'Dài','Rộng','Giá/m2']
    }

    for column in df.columns:
        if column in column_type['string']:
            df.loc[:, column] = df[column].fillna("Không xác định")
        # if column in column_type['date']:
            # df.loc[:, column] = pd.to_datetime(df[column],format="%Y-%m-%d", errors='coerce')
        
    #   if column in column_type['number']:
    #     df.loc[:, column] = df[column].fillna("0.0")
    #     # df.loc[:, column] = df[column].str.replace(',', '', regex=False).astype(float)
    
    remove_unit = {
        "Số phòng ngủ": [('phòng',''),('nhiều hơn 10', '11'),('Nhiều hơn 10', '11')], 
        "Diện tích": [('m²','')], 
        "Dài": [('m','')], 
        "Rộng": [('m','')], 
        "Giá/m2":  [(' triệu/m²','000000'),(' đ/m²','0'),(' tỷ/m²','000000000')], 
        "Số tầng":  [('nhiều hơn 10', '11'),('Nhiều hơn 10', '11')], 
        }

    for column in df.columns:
        # if column in ["Số phòng ngủ", 'Số tầng']:
        #   df.loc[:, column] = df[column].str.replace('nhiều hơn 10', '11', regex=False)
        #   df.loc[:, column] = df[column].str.replace('Nhiều hơn 10', '11', regex=False)
            
        if column in remove_unit:
            for patern, new_value in remove_unit[column]:
                df.loc[:, column] = df[column].str.replace(patern, new_value, regex=False)
            
    for column in column_type['number']:
        df.loc[:, column] = df[column].str.replace(',', '', regex=False).astype(float)
    
    # print(df)
        
    return df

In [27]:
df = preprocessing(df)
df.head()

,Ngày,Địa chỉ,Quận,Huyện,Loại hình nhà ở,Giấy tờ pháp lý,Số tầng,Số phòng ngủ,Diện tích,Dài,Rộng,Giá/m2
0,8/5/2020,"Đường Hoàng Quốc Việt, Phường Nghĩa Đô, Quận C...",Quận Cầu Giấy,Phường Nghĩa Đô,"Nhà ngõ, hẻm",Đã có sổ,4.0,5.0,46.0,NaN,NaN,8696000000.0
1,8/5/2020,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà mặt phố, mặt tiền",Không xác định,NaN,3.0,37.0,NaN,NaN,11622000000.0
2,8/5/2020,"phố minh khai, Phường Minh Khai, Quận Hai Bà T...",Quận Hai Bà Trưng,Phường Minh Khai,"Nhà ngõ, hẻm",Đã có sổ,4.0,4.0,40.0,10.0,4.0,65000000.0
3,8/5/2020,"Đường Võng Thị, Phường Thụy Khuê, Quận Tây Hồ,...",Quận Tây Hồ,Phường Thụy Khuê,"Nhà ngõ, hẻm",Đã có sổ,NaN,6.0,51.0,12.75,4.0,100000000.0
4,8/5/2020,"Đường Kim Giang, Phường Kim Giang, Quận Thanh ...",Quận Thanh Xuân,Phường Kim Giang,"Nhà ngõ, hẻm",Không xác định,NaN,4.0,36.0,9.0,4.0,8611000000.0


In [28]:
question = """
Mô tả tổng quan dữ liệu

"""

In [29]:
column_description = """
Ngày: Ngày mà thông tin về bất động sản được ghi nhận.
Địa chỉ: Địa chỉ chi tiết của bất động sản, bao gồm tên đường, phường/xã, quận/huyện.
Quận: Đơn vị hành chính cấp hành chính lớn hơn phường/xã, nơi bất động sản đặt tại.
Huyện: Đơn vị hành chính lớn hơn phường/xã nơi bất động sản đặt tại.
Loại hình nhà ở: Mô tả loại hình của bất động sản, ví dụ nhà ngõ, nhà mặt phố, biệt thự, chung cư, v.v.
Giấy tờ pháp lý: Trạng thái pháp lý của bất động sản, bao gồm những loại như đã có sổ đỏ, đang chờ cấp sổ, hoặc không rõ ràng.
Số tầng: Số tầng của bất động sản, bao gồm cả tầng hầm và tầng trệt nếu có.
Số phòng ngủ: Số lượng phòng ngủ trong bất động sản.
Diện tích: Diện tích tổng thể của bất động sản, tính bằng mét vuông (m²).
Dài: Chiều dài của bất động sản, tính bằng mét (m).
Rộng: Chiều rộng của bất động sản, tính bằng mét (m).
Giá/m2: Giá bán hoặc giá cho thuê trung bình của một mét vuông bất động sản, tính bằng đơn vị tiền tệ (ví dụ như VND).
"""

In [30]:
def question_llm(summary_prompt):
  llm_answer = llm([HumanMessage(content=summary_prompt)])
  if '```json' in llm_answer.content:
      match = re.search(r'```json\n(.*?)```', llm_answer.content, re.DOTALL)
      if match: json_str = match.group(1)
  else: json_str = llm_answer.content
  print(json_str)
  return json.loads(json_str)

In [31]:
related_template = """
  Bạn là một data analyst.  Shape của data frame là {shape_info}.  Giá trị head(5) của data frame là: {head_info}. Giá trị nunique() của data frame là: {nunique_info}. Mô tả của data frame là: {description_info}. Mô tả các cột là: {column_description}. Câu hỏi của người dùng là: {question}. Data đã được làm sạch và tiền xử lý, các null đã được lấp đầy và mã hoá để sẵn sàng để huấn luyện mô hình máy học.  Dựa vào thông tin data cung cấp, hãy giúp tôi quyết định rằng câu hỏi có liên quan đến visualization và có thể visualization trên dữ liệu data frame giá nhà được cung cấp không. Các tùy chọn là: true:Có, false:Không.  Chỉ dữ liệu được trả về ở định dạng json mà không có bất kỳ lời giải thích hoặc nội dung nào khác. Phản hồi mẫu: {{"isRelated":true}}
"""

In [32]:
attrs_template = """  Bạn là một data analyst.  Shape của data frame là {shape_info}.  Giá trị head(5) của data frame là: {head_info}. Giá trị nunique() của data frame là: {nunique_info}. Mô tả của data frame là: {description_info}. Mô tả các cột là: {column_description}. Câu hỏi của người dùng là: {question}. Data đã được làm sạch và tiền xử lý, các null đã được lấp đầy và mã hoá để sẵn sàng để huấn luyện mô hình máy học. Dựa vào thông tin data cung cấp, hãy giúp tôi quyết định rằng câu hỏi có liên quan đến các trường dữ liệu nào trên dữ liệu data frame giá nhà được cung cấp. Các trường tùy chọn là: ['Ngày', 'Địa chỉ', 'Quận', 'Huyện', 'Loại hình nhà ở','Giấy tờ pháp lý', 'Số tầng', 'Số phòng ngủ', 'Diện tích', 'Dài','Rộng', 'Giá/m2'] Chỉ dữ liệu được trả về ở định dạng json mà không có bất kỳ lời giải thích hoặc nội dung nào khác. Phản hồi mẫu: {{"attrs":["Ngày", "Địa chỉ"], "reason": ""}}

"""

In [33]:
single_attr_chart_visualization_template = """ Bạn là một data analyst.  Shape của data frame là {shape_info}.  Giá trị head(5) của data frame là: {head_info}. Giá trị nunique() của data frame là: {nunique_info}. Mô tả của data frame là: {description_info}. Mô tả các cột là: {column_description}.Các trường cần visusalization là: {attrs}. Câu hỏi của người dùng là: {question}. Data đã được làm sạch và tiền xử lý, các null đã được lấp đầy và mã hoá để sẵn sàng để huấn luyện mô hình máy học. Dựa vào thông tin data cung cấp, hãy giúp tôi chọn duy nhất một biểu đồ tốt nhất để visualization trên dữ liệu data frame giá nhà được cung cấp. Các biểu đồ tùy chọn là: ['Donut chart', 'Violin plot', 'Distribution histogram', 'Boxplot', 'Density plot', 'Strip plot', 'Distribution boxplot'] Chỉ dữ liệu được trả về ở định dạng json mà không có bất kỳ lời giải thích hoặc nội dung nào khác. Phản hồi mẫu: {{"chart":["Donut chart"], "reason": ""}}"""

  



multiple_attr_chart_visualization_template = """ Bạn là một data analyst.  Shape của data frame là {shape_info}.  Giá trị head(5) của data frame là: {head_info}. Giá trị nunique() của data frame là: {nunique_info}. Mô tả của data frame là: {description_info}. Mô tả các cột là: {column_description}.Các trường cần visusalization là: {attrs}. Câu hỏi của người dùng là: {question}. Data đã được làm sạch và tiền xử lý, các null đã được lấp đầy và mã hoá để sẵn sàng để huấn luyện mô hình máy học. Dựa vào thông tin data cung cấp, hãy giúp tôi chọn duy nhất một biểu đồ tốt nhất để visualization trên dữ liệu data frame giá nhà được cung cấp. Các biểu đồ tùy chọn là: ["Violin plot", "Boxplot", "Heatmap", "Strip plot", "Line plot", "Scatter plot"] Chỉ dữ liệu được trả về ở định dạng json mà không có bất kỳ lời giải thích hoặc nội dung nào khác. Phản hồi mẫu: {{"chart":["Violin plot"], "reason": ""}}"""

In [34]:
shape_info, head_info, nunique_info, description_info = get_data_overview(df)

In [35]:
def predict_visualization_related_question():
  related_prompt_template = PromptTemplate(input_variables=["shape_info", "head_info", "nunique_info", "description_info", "question", "column_description"], template=related_template)

  related_summary_prompt = related_prompt_template.format(shape_info=shape_info, head_info=head_info, nunique_info=nunique_info, description_info=description_info,column_description=column_description ,question=question)

  related_value = question_llm(related_summary_prompt)
  return related_value

In [36]:
def predict_visualization_related_attrs():
  attrs_prompt_template = PromptTemplate(input_variables=["shape_info", "head_info", "nunique_info", "description_info", "question", "column_description"], template=attrs_template)
  attrs_summary_prompt = attrs_prompt_template.format(shape_info=shape_info, head_info=head_info, nunique_info=nunique_info, description_info=description_info,column_description=column_description ,question=question)

  attrs_value = question_llm(attrs_summary_prompt)
  return attrs_value

In [37]:
def predict_visualization_chart(attrs_value,template):
  prompt_template = PromptTemplate(input_variables=["shape_info", "head_info", "nunique_info", "description_info", "question", "column_description","attrs"], template=template)
  summary_prompt = prompt_template.format(shape_info=shape_info, head_info=head_info, nunique_info=nunique_info, description_info=description_info,column_description=column_description , attrs=attrs_value["attrs"], question=question)

  value = question_llm(summary_prompt)
  return value

In [38]:
related_value = predict_visualization_related_question()

if related_value['isRelated'] == True:
  attrs_value = predict_visualization_related_attrs()
  
  if len(attrs_value['attrs']) > 1:
    chart  = predict_visualization_chart(attrs_value, multiple_attr_chart_visualization_template)
    
  else:
    chart  = predict_visualization_chart(attrs_value, single_attr_chart_visualization_template)
else:
  print("Dữ liệu không liên quan đến giá nhà")

{"isRelated":true}
{"attrs":["Ngày", "Địa chỉ", "Quận", "Huyện", "Loại hình nhà ở","Giấy tờ pháp lý", "Số tầng", "Số phòng ngủ", "Diện tích", "Dài","Rộng", "Giá/m2"], "reason": "Tất cả các trường dữ liệu trong data frame đều có thể liên quan đến việc quyết định giá nhà, vì vậy chúng ta nên xem xét tất cả các trường dữ liệu để đảm bảo không bỏ sót thông tin quan trọng nào."}
{
  "chart": ["Violin plot", "Boxplot", "Heatmap", "Strip plot", "Line plot", "Scatter plot"],
  "reason": "Dữ liệu của chúng ta bao gồm nhiều biến liên tục như diện tích, giá/m2, số tầng, số phòng ngủ, chiều dài và chiều rộng. Do đó, để hiểu rõ phân phối của các biến này và tìm ra các outlier, chúng ta nên sử dụng Violin plot và Boxplot."
}
